In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [3]:
# Load the CICIDS 2017 dataset
data = pd.read_csv('Wednesday-workingHours.pcap_ISCX.csv')

In [7]:
# Preprocessing
# Encode categorical labels
label_encoder = LabelEncoder()
data[' Label'] = label_encoder.fit_transform(data[' Label'])

In [8]:
# Split features and labels
X = data.drop(columns=[' Label'])
y = data[' Label']

In [9]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [11]:
cnn_model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [12]:
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
cnn_model.fit(X_train[..., np.newaxis], y_train, epochs=5, batch_size=64, validation_split=0.1)


KeyError: (Ellipsis, None)

In [ ]:
# Train a Random Forest model using CNN predictions as features
cnn_train_features = cnn_model.predict(X_train[..., np.newaxis])
cnn_test_features = cnn_model.predict(X_test[..., np.newaxis])

In [ ]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(cnn_train_features, y_train)

In [ ]:
# Evaluate the integrated model
rf_cnn_predictions = rf_model.predict(cnn_test_features)
cnn_rf_accuracy = accuracy_score(y_test, rf_cnn_predictions)
print("Accuracy of the integrated model (Random Forest + CNN):", cnn_rf_accuracy)